In [1]:
import numpy as np
import pandas as pd
import warnings
import os
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime,date

### 定義用到的函數
1. turn_date_time：將string轉為date
2. isfloat:判斷是否為float

In [2]:
def turn_date_time(string):
    string = string.split('/')
    da = date(2021,int(string[1]),int(string[2]))
    return da

def isfloat(num):
    try:
        float(num)
        return True
    except ValueError:
        return False

### 讀取資料並處理資料格式

In [3]:
hsinchu = pd.read_csv('新竹_2021.csv')
hsinchu = hsinchu.drop(index=0)
hsinchu = hsinchu.rename({'測站                  ':'station','日期                  ':'date','測項                  ':'item'}, axis=1)
hsinchu['date'] = hsinchu['date'].apply(lambda x: x.split())
hsinchu['date']= hsinchu['date'].apply(lambda x: x[0])
hsinchu['date']= hsinchu['date'].apply(lambda x: turn_date_time(x))
times = ['0', '1', '2', '3', '4', '5', '6', '7', '8',
       '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
       '21', '22', '23']

hsinchu['item'] = hsinchu['item'].apply(lambda x: x.split()[0])

### 取出10~12月資料

In [4]:
start_date = date(2021, 10, 1)
hsinchu = hsinchu[hsinchu['date'] >= start_date]
hsinchu = hsinchu.reset_index(inplace=False).drop('index',axis=1)

### 填缺值

In [5]:
df = pd.DataFrame()

for i in times:
    df[i] = hsinchu[i].map(isfloat)

In [6]:
df

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1652,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1653,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1654,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [7]:
for r in df.columns:
    n = int(r)
    for j in range(1656):
        if df.iloc[j,n] == False:
            k=n-1
            while k>=0 and df.iloc[j,k]!= True:
                k-=1
            l=n+1
            while l<24 and df.iloc[j,l]!=True:
                l+=1
            
            if k>=0 and l<24:
                hsinchu.iloc[j,n+3] = (float(hsinchu.iloc[j,k+3]) + float(hsinchu.iloc[j,l+3]))/2
            elif k<0 and l<24:
                hsinchu.iloc[j,n+3] = (0+ float(hsinchu.iloc[j,l+3]))/2
            elif k>=0 and l>24:
                hsinchu.iloc[j,n+3] = (float(hsinchu.iloc[j,k+3]) + 0)/2
            else:
                hsinchu.iloc[j,n+3] = 0
            df.iloc[j,n] = True

### 製作時序資料

In [8]:
pollutants = hsinchu['item'].unique()
pollutants

array(['AMB_TEMP', 'CH4', 'CO', 'NMHC', 'NO', 'NO2', 'NOx', 'O3', 'PM10',
       'PM2.5', 'RAINFALL', 'RH', 'SO2', 'THC', 'WD_HR', 'WIND_DIREC',
       'WIND_SPEED', 'WS_HR'], dtype=object)

In [9]:
hours = 61*24
data = pd.DataFrame()

In [10]:
for i,it in enumerate(pollutants):
    data[it]=''
    tmp = hsinchu[hsinchu['item']==it]
    total_ar=[]
    for t in range(len(tmp)):
        ar = list(tmp.iloc[t,:])
        del ar[0:3]
        for a in ar:
            total_ar.append(float(a))
    data[it] = total_ar

In [11]:
data

,AMB_TEMP,CH4,CO,NMHC,NO,NO2,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,THC,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR
0,28.3,2.04,0.34,0.17,0.9,18.8,19.8,16.0,28.0,28.0,0.0,71.0,1.8,2.21,62.0,33.0,0.8,0.7
1,28.3,2.02,0.30,0.13,0.2,11.9,12.2,21.5,24.0,22.0,0.0,66.0,1.0,2.15,121.0,183.0,1.2,0.6
2,27.8,2.12,0.30,0.12,0.5,15.1,15.6,16.9,29.0,26.0,0.0,76.0,1.0,2.24,164.0,160.0,1.1,0.6
3,27.8,2.18,0.29,0.14,0.4,12.8,13.2,16.4,32.0,24.0,0.0,79.0,1.3,2.32,156.0,151.0,0.6,0.4
4,27.6,2.19,0.30,0.17,0.2,14.9,15.1,12.6,31.0,28.0,0.0,81.0,1.5,2.36,110.0,90.0,0.9,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203,15.8,2.01,0.29,0.09,0.7,11.6,12.3,32.1,19.0,13.0,0.0,71.0,1.3,2.10,53.0,55.0,2.2,2.0
2204,15.6,2.04,0.33,0.11,0.8,13.6,14.4,29.6,18.0,10.0,0.0,74.0,1.1,2.15,47.0,41.0,1.7,1.6
2205,15.7,2.03,0.32,0.10,0.7,13.6,14.4,30.3,8.0,11.0,0.0,78.0,1.4,2.13,37.0,36.0,2.5,2.0
2206,15.9,2.01,0.28,0.08,0.5,11.8,12.3,31.9,10.0,9.0,0.0,79.0,1.6,2.09,42.0,53.0,2.3,2.1


### 將資料切割成訓練集(10.11月)以及測試集(12月)

In [12]:
training = data.iloc[0:1464,:]
testing = data.iloc[1464:,:]
testing = testing.reset_index(drop=True)

In [13]:
training

,AMB_TEMP,CH4,CO,NMHC,NO,NO2,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,THC,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR
0,28.3,2.04,0.34,0.17,0.9,18.8,19.8,16.0,28.0,28.0,0.0,71.0,1.8,2.21,62.0,33.0,0.8,0.7
1,28.3,2.02,0.30,0.13,0.2,11.9,12.2,21.5,24.0,22.0,0.0,66.0,1.0,2.15,121.0,183.0,1.2,0.6
2,27.8,2.12,0.30,0.12,0.5,15.1,15.6,16.9,29.0,26.0,0.0,76.0,1.0,2.24,164.0,160.0,1.1,0.6
3,27.8,2.18,0.29,0.14,0.4,12.8,13.2,16.4,32.0,24.0,0.0,79.0,1.3,2.32,156.0,151.0,0.6,0.4
4,27.6,2.19,0.30,0.17,0.2,14.9,15.1,12.6,31.0,28.0,0.0,81.0,1.5,2.36,110.0,90.0,0.9,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,20.3,2.10,0.46,0.11,1.1,12.5,13.7,34.9,58.0,36.0,0.0,65.0,2.2,2.21,58.0,57.0,3.8,3.4
1460,19.9,2.10,0.45,0.10,1.0,10.5,11.5,33.5,46.0,34.0,0.0,62.0,2.2,2.20,56.0,54.0,3.8,3.5
1461,19.4,2.09,0.42,0.09,1.1,8.8,10.0,33.8,52.0,32.0,0.0,58.0,2.1,2.18,47.0,35.0,4.0,3.6
1462,19.0,2.07,0.37,0.08,0.9,8.2,9.2,32.6,54.0,32.0,0.0,56.0,2.1,2.15,45.0,56.0,4.4,3.8


In [14]:
training.describe()

,AMB_TEMP,CH4,CO,NMHC,NO,NO2,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,THC,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR
count,1464.000000,1464.000000,1464.000000,1464.000000,1464.000000,1464.000000,1464.000000,1464.000000,1464.000000,1464.000000,1464.000000,1464.000000,1464.000000,1464.000000,1464.000000,1464.000000,1464.000000,1464.000000
mean,24.433197,2.028610,0.299024,0.101902,1.522908,12.437901,14.005575,34.141583,23.126537,12.131660,0.055669,67.689891,1.417701,2.130482,80.963115,81.742828,2.340130,1.952869
std,4.294394,0.091031,0.115699,0.062425,2.764122,5.606925,7.016187,15.692564,11.694728,6.574365,0.352913,13.855992,0.823269,0.138308,69.102676,70.714764,1.291225,1.093612
min,14.300000,1.010000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,30.000000,0.000000,1.005000,3.000000,0.000000,0.300000,0.000000
25%,21.400000,1.987500,0.230000,0.060000,0.575000,8.500000,9.500000,23.600000,15.000000,8.000000,0.000000,57.000000,0.800000,2.050000,48.000000,47.000000,1.300000,1.000000
50%,23.900000,2.010000,0.280000,0.090000,0.900000,11.500000,12.700000,34.550000,21.000000,11.000000,0.000000,67.000000,1.300000,2.110000,55.000000,56.000000,2.200000,1.900000
75%,27.500000,2.050000,0.350000,0.130000,1.500000,15.100000,16.600000,42.725000,29.000000,15.000000,0.000000,80.000000,1.900000,2.180000,71.000000,75.000000,3.200000,2.700000
max,36.200000,3.060000,1.220000,0.500000,44.200000,43.000000,63.500000,94.700000,91.000000,43.000000,5.500000,97.000000,4.700000,3.470000,358.000000,358.000000,6.500000,5.400000


In [15]:
testing.describe()

,AMB_TEMP,CH4,CO,NMHC,NO,NO2,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,THC,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR
count,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.00000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000
mean,18.478495,2.043198,0.332424,0.106789,1.993053,12.986828,15.025521,30.742364,25.154570,14.236559,0.09459,71.002688,1.285223,2.149987,64.084677,66.973118,2.844489,2.404570
std,2.670056,0.076970,0.144586,0.069505,4.319513,6.678750,9.548368,10.979649,13.875054,7.649001,0.53908,15.764672,0.538400,0.136023,52.617153,56.519336,1.497714,1.242466
min,12.300000,1.940000,0.140000,0.010000,0.000000,3.000000,3.600000,0.000000,0.000000,0.000000,0.00000,33.000000,0.000000,1.970000,6.000000,2.000000,0.300000,0.100000
25%,16.775000,2.000000,0.250000,0.060000,0.700000,8.500000,9.500000,25.600000,15.000000,9.000000,0.00000,58.000000,0.900000,2.060000,43.000000,43.000000,1.600000,1.500000
50%,18.400000,2.030000,0.300000,0.090000,1.000000,11.300000,12.650000,32.950000,23.000000,12.000000,0.00000,73.000000,1.200000,2.110000,50.000000,52.000000,2.900000,2.500000
75%,20.100000,2.060000,0.370000,0.130000,1.800000,15.600000,17.200000,37.700000,33.000000,17.250000,0.00000,84.250000,1.600000,2.190000,57.000000,60.000000,3.900000,3.325000
max,27.100000,2.500000,1.450000,0.560000,49.200000,45.400000,77.500000,71.100000,84.000000,50.000000,6.50000,97.000000,3.900000,3.060000,349.000000,359.000000,7.100000,5.300000


# Time Series

### 將未來第一個小時當預測目標，製作出x&y資料

In [16]:
def slice_six(df):
    m = len(df)
    dif = 6
    X = np.empty([m-dif,18*6],dtype=float)
    X_PM = np.empty([m-dif,6],dtype=float)
    Y = np.empty([m-dif,1],dtype=float)
    for i in range(m-dif):
        a = []
        b = []
        for j in range(6):
            for k in range(18):
                a.append(df.iloc[i+j,k])
            b.append(df['PM2.5'][i+j])
        X[i,:] = a
        X_PM[i,:]=b
        Y[i,0] = df['PM2.5'][i+dif]
    return (X,X_PM,Y)
training_X, training_X_PM,training_y = slice_six(training)
testing_X, testing_X_PM, testing_y = slice_six(testing)

### 將未來第六個小時當預測目標，製作出x&y資料

In [17]:
def slice_six_6(df):
    m = len(df)
    dif = 6
    X = np.empty([m-dif-5,18*6],dtype=float)
    X_PM = np.empty([m-dif-5,6],dtype=float)
    Y = np.empty([m-dif-5,1],dtype=float)
    for i in range(m-dif-5):
        a = []
        b = []
        for j in range(6):
            for k in range(18):
                a.append(df.iloc[i+j,k])
            b.append(df['PM2.5'][i+j])
        X[i,:] = a
        X_PM[i,:]=b
        Y[i,0] = df['PM2.5'][i+dif+5]
    return (X,X_PM,Y)
training_X_6, training_X_PM_6,training_y_6 = slice_six_6(training)
testing_X_6, testing_X_PM_6, testing_y_6 = slice_six_6(testing)

# Linear Regression

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [19]:
model = LinearRegression().fit(training_X, training_y)
y_predict = model.predict(testing_X)
print("The MAE of forecasting next hour by Linear Regression with 18 features:",mean_absolute_error(testing_y, y_predict))

model = LinearRegression().fit(training_X_PM, training_y)
y_predict = model.predict(testing_X_PM)
print("The MAE of forecasting next hour by Linear Regression with 1 features:",mean_absolute_error(testing_y, y_predict))

model = LinearRegression().fit(training_X_6, training_y_6)
y_predict = model.predict(testing_X_6)
print("The MAE of forecasting next six hours by Linear Regression with 18 features:",mean_absolute_error(testing_y_6, y_predict))

model = LinearRegression().fit(training_X_PM_6, training_y_6)
y_predict = model.predict(testing_X_PM_6)
print("The MAE of forecasting next six hours by Linear Regression with 1 features:",mean_absolute_error(testing_y_6, y_predict))

The MAE of forecasting next hour by Linear Regression with 18 features: 2.6465815271938715
The MAE of forecasting next hour by Linear Regression with 1 features: 2.67859512930202
The MAE of forecasting next six hours by Linear Regression with 18 features: 4.257704035956232
The MAE of forecasting next six hours by Linear Regression with 1 features: 4.307073329319159


# XGBoost

In [20]:
from xgboost import XGBRegressor

In [21]:
reg = XGBRegressor(n_estimators=1000).fit(training_X, training_y)
y_predict = reg.predict(testing_X)
print("The MAE of forecasting next hour by XGBoosting with 18 features:",mean_absolute_error(testing_y, y_predict))

reg = XGBRegressor(n_estimators=1000).fit(training_X_PM, training_y)
y_predict = reg.predict(testing_X_PM)
print("The MAE of forecasting next hour by XGBoosting with 1 features:",mean_absolute_error(testing_y, y_predict))

reg = XGBRegressor(n_estimators=1000).fit(training_X_6, training_y_6)
y_predict = reg.predict(testing_X_6)
print("The MAE of forecasting next six hours by XGBoosting with 18 features:",mean_absolute_error(testing_y_6, y_predict))

reg = XGBRegressor(n_estimators=1000).fit(training_X_PM_6, training_y_6)
y_predict = reg.predict(testing_X_PM)
print("The MAE of forecasting next hour by XGBoosting with 1 features:",mean_absolute_error(testing_y, y_predict))

The MAE of forecasting next hour by XGBoosting with 18 features: 3.0743585204690453
The MAE of forecasting next hour by XGBoosting with 1 features: 3.2265741151320904
The MAE of forecasting next six hours by XGBoosting with 18 features: 5.138281385700179
The MAE of forecasting next hour by XGBoosting with 1 features: 4.243214797198288


# Normalization

In [22]:
def nor_array(array):
    mean = np.mean(array, axis=0)
    std = np.std(array, axis=0)
    m,n = array.shape
    nor_array = np.empty([m,n],dtype=float)
    for i in range(m):
        for j in range(n):
            nor_array[i][j] = (array[i][j]-mean[j])/std[j]
    return nor_array

In [23]:
nor_training_X = nor_array(training_X)
nor_testing_X = nor_array(testing_X)
nor_training_X_PM = nor_array(training_X_PM)
nor_testing_X_PM = nor_array(testing_X_PM)
nor_training_X_6 = nor_array(training_X_6)
nor_testing_X_6 = nor_array(testing_X_6)
nor_training_X_PM_6 = nor_array(training_X_PM_6)
nor_testing_X_PM_6 = nor_array(testing_X_PM_6)

In [24]:
model = LinearRegression().fit(nor_training_X, training_y)
y_predict = model.predict(nor_testing_X)
print("The MAE of forcasting next hour by Linear Regression with 18 features:",mean_absolute_error(testing_y, y_predict))

model = LinearRegression().fit(nor_training_X_PM, training_y)
y_predict = model.predict(nor_testing_X_PM)
print("The MAE of forcasting next hour by Linear Regression with 1 features:",mean_absolute_error(testing_y, y_predict))

model = LinearRegression().fit(nor_training_X_6, training_y_6)
y_predict = model.predict(nor_testing_X_6)
print("The MAE of forcasting next six hours by Linear Regression with 18 features:",mean_absolute_error(testing_y_6, y_predict))

model = LinearRegression().fit(nor_training_X_PM_6, training_y_6)
y_predict = model.predict(nor_testing_X_PM_6)
print("The MAE of forcasting next six hours by Linear Regression with 1 features:",mean_absolute_error(testing_y_6, y_predict))

The MAE of forcasting next hour by Linear Regression with 18 features: 4.571960955459041
The MAE of forcasting next hour by Linear Regression with 1 features: 3.100893268105069
The MAE of forcasting next six hours by Linear Regression with 18 features: 9.714286175157392
The MAE of forcasting next six hours by Linear Regression with 1 features: 4.391246544609768


In [25]:
reg = XGBRegressor(n_estimators=1000).fit(nor_training_X, training_y)
y_predict = reg.predict(nor_testing_X)
print("The MAE of forcasting next hour by XGBoosting with 18 features:",mean_absolute_error(testing_y, y_predict))

reg = XGBRegressor(n_estimators=1000).fit(nor_training_X_PM, training_y)
y_predict = reg.predict(nor_testing_X_PM)
print("The MAE of forcasting next hour by XGBoosting with 1 features:",mean_absolute_error(testing_y, y_predict))

reg = XGBRegressor(n_estimators=1000).fit(nor_training_X_6, training_y_6)
y_predict = reg.predict(nor_testing_X_6)
print("The MAE of forcasting next six hours by XGBoosting with 18 features:",mean_absolute_error(testing_y_6, y_predict))

reg = XGBRegressor(n_estimators=1000).fit(nor_training_X_PM_6, training_y_6)
y_predict = reg.predict(nor_testing_X_PM)
print("The MAE of forcasting next hour by XGBoosting with 1 features:",mean_absolute_error(testing_y, y_predict))

The MAE of forcasting next hour by XGBoosting with 18 features: 3.2661740017454153
The MAE of forcasting next hour by XGBoosting with 1 features: 3.4456792509329675
The MAE of forcasting next six hours by XGBoosting with 18 features: 4.870431038399835
The MAE of forcasting next hour by XGBoosting with 1 features: 4.701657292804097
